<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB01_Visualisation_tokenisation_embeddings_open_source.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Lab 1: Visualisation de la tokenisation et des embeddings (Version Open Source)
---

**Objectif:**
- Comprendre comment un texte est segmenté en tokens et converti en vecteurs numériques (embeddings).
- Apprendre à visualiser et interpréter ces structures.
- **Utiliser uniquement des modèles open source gratuits !**

**Durée estimée:**
- 90–120 minutes

**Livrable :**
- Notebook avec graphiques présentant la segmentation en tokens et une visualisation 2D de vecteurs numériques.

---

## Step 1: Pré-requis (5 min)

Installation des librairies nécessaires :

In [ ]:
!pip install transformers sentence-transformers matplotlib scikit-learn pandas numpy scipy networkx -q

---
## Step 2: Inspection de la tokenisation (15 min)

Utilisation de `transformers` et `AutoTokenizer` pour découper le texte en tokens :

In [ ]:
from transformers import AutoTokenizer

# Initialize tokenizer with a popular open-source model (DistilBERT)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Sample text
sample_text = "Agentic AI agents can plan, reason, and use tools."

# Tokenize
tokens = tokenizer.encode(sample_text)
tokens_decoded = tokenizer.tokenize(sample_text)

# Display results
print("="*70)
print(f"Texte originel: {sample_text}")
print(f"Nombre de tokens: {len(tokens)}")
print(f"Token IDs: {tokens}")
print("\nTokens décodés (individuellement):")
for i, token in enumerate(tokens_decoded, 1):
    print(f" {i}. '{token}'")
print("="*70)

### Expérimentez avec différents types de texte

Regardons comment la ponctuation, les espaces, et les caractères spéciaux affectent la tokenisation :

In [ ]:
test_texts = [
    "Hello, World!",
    "HelloWorld",
    "hello world",
    "AI is 🔥.",
    "def my_function():\n return True",
    "AGENTIC",
    "agentic",
]

print("\n" + "="*70)
print("Analyse de la Tokenisation sur différents exemples de texte")
print("="*70)

for text in test_texts:
    tokens = tokenizer.encode(text)
    decoded = tokenizer.tokenize(text)
    print(f"\nText: {repr(text)}")
    print(f"Tokens: {len(tokens)} → {tokens}")
    print(f"Decoded: {decoded}")

---
## Step 3 : Comparaison de la longueur des jetons dans les textes (10 min)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

sentences = [
    "AI is amazing.",
    "Artificial Intelligence is amazing.",
    "AI is 🔥.",
    "The quick brown fox jumps over the lazy dog.",
    "The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog.",
]

# Collect data
data = []
for sentence in sentences:
    token_list = tokenizer.encode(sentence)
    data.append({
        "Text": sentence,
        "Character Count": len(sentence),
        "Token Count": len(token_list),
        "Ratio (Chars/Token)": len(sentence) / len(token_list)
    })

df = pd.DataFrame(data)
print("\n" + "="*100)
print("Comparaison de la longueur des Tokens")
print("="*100)
print(df.to_string(index=False))
print("="*100)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart: Character vs Token count
x = range(len(df))
axes[0].bar([i - 0.2 for i in x], df['Character Count'], width=0.4, label='Characters', color='steelblue')
axes[0].bar([i + 0.2 for i in x], df['Token Count'], width=0.4, label='Tokens', color='coral')
axes[0].set_xlabel('Sentence Index', fontsize=11, fontweight='bold')
axes[0].set_ylabel('Count', fontsize=11, fontweight='bold')
axes[0].set_title('Character Count vs Token Count', fontsize=12, fontweight='bold')
axes[0].set_xticks(x)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Line chart: Char/Token ratio
axes[1].plot(x, df['Ratio (Chars/Token)'], marker='o', linewidth=2, markersize=8, color='green')
axes[1].set_xlabel('Sentence Index', fontsize=11, fontweight='bold')
axes[1].set_ylabel('Ratio (Chars/Token)', fontsize=11, fontweight='bold')
axes[1].set_title('Character-to-Token Ratio', fontsize=12, fontweight='bold')
axes[1].set_xticks(x)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Remarque : Les caractères spéciaux ont tendance à utiliser plus de jetons par caractère !")

---
## Step 4: Générer des embeddings (20 min)

Utilisons le modèle open source `sentence-transformers` pour créer des vecteurs numériques :

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load a pre-trained model (free and open-source)
# Options: 'all-MiniLM-L6-v2', 'all-mpnet-base-v2', 'distiluse-base-multilingual-cased-v2'
model = SentenceTransformer('all-MiniLM-L6-v2')

# Texts to embed
texts = [
    "Agentic AI",
    "Autonomous agents",
    "Bananas are yellow",
    "Machine learning models",
    "Fruit is delicious",
]

embeddings = []

print("\n" + "="*70)
print("GENERATING EMBEDDINGS")
print("="*70)

for text in texts:
    vector = model.encode(text)
    embeddings.append(vector)
    print(f"✓ '{text}' → Vector length: {len(vector)}")

print("="*70)
print(f"\n📊 Embedding Dimension: {len(embeddings[0])}")
print(f"Number of texts embedded: {len(embeddings)}")

# Show a sample embedding (first 20 values)
print(f"\nSample embedding (first 20 values) for '{texts[0]}':")
print(embeddings[0][:20])

---
## Step 5: Réduction de dimensionnalité et visualisation (25 min)

Réduire les Embeddings à 2D pour permettre la visualisation :

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Convert to numpy array
embeddings_array = np.array(embeddings)

# Apply PCA for dimensionality reduction
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings_array)

# Get explained variance
explained_var = pca.explained_variance_ratio_

print("\n📈 PCA Explained Variance:")
print(f" PC1: {explained_var[0]:.4f} ({explained_var[0]*100:.2f}%)")
print(f" PC2: {explained_var[1]:.4f} ({explained_var[1]*100:.2f}%)")
print(f" Total: {sum(explained_var):.4f} ({sum(explained_var)*100:.2f}%)")

# Visualization
plt.figure(figsize=(10, 8))

colors = ['#FF6B6B', '#FF6B6B', '#4ECDC4', '#4ECDC4', '#4ECDC4']

for i, (txt, color) in enumerate(zip(texts, colors)):
    x, y = embeddings_2d[i]
    plt.scatter(x, y, s=300, alpha=0.7, color=color, edgecolors='black', linewidth=2)
    plt.text(x + 0.05, y + 0.05, txt, fontsize=10, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.3', facecolor=color, alpha=0.3))

plt.xlabel(f'PC1 ({explained_var[0]*100:.1f}%)', fontsize=12, fontweight='bold')
plt.ylabel(f'PC2 ({explained_var[1]*100:.1f}%)', fontsize=12, fontweight='bold')
plt.title('Embedding Visualization (PCA 2D)', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.axhline(y=0, color='k', linewidth=0.5)
plt.axvline(x=0, color='k', linewidth=0.5)
plt.tight_layout()
plt.show()

print("\n💡 Remarque : Les textes sémantiquement similaires se retrouvent proches les uns des autres !")

### Explorateur interactif d'embeddings

Ajoutons d'autres phrases et visualisons les groupes :

In [ ]:
# Additional texts
additional_texts = [
    "Intelligent systems planning",
    "Orange is a fruit",
    "Deep learning networks",
]

# Combine all texts
all_texts = texts + additional_texts

print("\nGenerating embeddings for extended set...")
all_embeddings = [model.encode(text) for text in all_texts]

# Apply PCA
all_embeddings_array = np.array(all_embeddings)
pca_extended = PCA(n_components=2)
all_embeddings_2d = pca_extended.fit_transform(all_embeddings_array)

explained_var_extended = pca_extended.explained_variance_ratio_

# Visualization
plt.figure(figsize=(12, 9))

# Extended colors
colors_extended = ['#FF6B6B', '#FF6B6B', '#4ECDC4', '#4ECDC4', '#4ECDC4', '#FF6B6B', '#4ECDC4', '#FF6B6B']

for i, (txt, color) in enumerate(zip(all_texts, colors_extended)):
    x, y = all_embeddings_2d[i]
    plt.scatter(x, y, s=350, alpha=0.7, color=color, edgecolors='black', linewidth=2)
    plt.text(x + 0.08, y + 0.08, txt, fontsize=9, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.3', facecolor=color, alpha=0.3))

plt.xlabel(f'PC1 ({explained_var_extended[0]*100:.1f}%)', fontsize=12, fontweight='bold')
plt.ylabel(f'PC2 ({explained_var_extended[1]*100:.1f}%)', fontsize=12, fontweight='bold')
plt.title('Extended Embedding Visualization (PCA 2D)', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.axhline(y=0, color='k', linewidth=0.5)
plt.axvline(x=0, color='k', linewidth=0.5)
plt.tight_layout()
plt.show()

print(f"\n📊 Extended PCA Explained Variance: {sum(explained_var_extended)*100:.2f}%")

---
## Step 6: Classe utilitaire pour analyse complète

Créons une classe réutilisable pour analyser n'importe quel texte :

In [ ]:
from scipy.spatial.distance import cosine

class EmbeddingAnalyzer:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.reference_embedding = None

    def set_reference(self, text):
        """Set reference text for similarity comparison"""
        self.reference_embedding = self.model.encode(text)

    def compute_similarity(self, embedding):
        """Compute cosine similarity with reference embedding"""
        if self.reference_embedding is None:
            return None
        return 1 - cosine(self.reference_embedding, embedding)

    def display_analysis(self, text):
        """Display complete analysis for a text"""
        # Tokenization
        tokens = self.tokenizer.encode(text)
        tokens_decoded = self.tokenizer.tokenize(text)

        # Embedding
        embedding = self.model.encode(text)

        # Similarity
        similarity = self.compute_similarity(embedding)

        # Display
        print("\n" + "="*80)
        print(f"TEXTE: '{text}'")
        print("="*80)
        print(f"Nombre de caractères : {len(text)}")
        print(f"Nombre de tokens : {len(tokens)}")
        print(f"Dimension Embedding : {len(embedding)}")

        if similarity is not None:
            print(f"\nSimilarité avec la Reference : {similarity:.4f}")

        print(f"\nTokens:")
        for i, token in enumerate(tokens_decoded, 1):
            print(f" {i}. {token}")

        print(f"\nPrévisualisation des Embeddings (10 premières valeurs): {embedding[:10].tolist()}")
        print("="*80)

# Initialize analyzer
analyzer = EmbeddingAnalyzer(model, tokenizer)
analyzer.set_reference("AI and agents")

# Test with custom texts
custom_texts = [
    "Autonomous intelligent systems",
    "Yellow bananas and fruits",
    "Neural network architectures",
]

for text in custom_texts:
    analyzer.display_analysis(text)

---
## Step 7: Visualisation avancée - Réseau de similarité

Créons un graphique de réseau illustrant les relations de similarité :

In [ ]:
import matplotlib.patches as mpatches
import networkx as nx

# Prepare all texts for analysis
network_texts = all_texts[:8]  # Use first 8 texts

# Generate embeddings for all texts
network_embeddings = [model.encode(text) for text in network_texts]

# Compute similarity matrix
similarity_matrix = []
for emb1 in network_embeddings:
    row = []
    for emb2 in network_embeddings:
        sim = 1 - cosine(emb1, emb2)
        row.append(sim)
    similarity_matrix.append(row)

similarity_matrix = np.array(similarity_matrix)

# Create network graph
G = nx.Graph()

# Add nodes
for i, text in enumerate(network_texts):
    G.add_node(i, label=text)

# Add edges for high similarity (>0.5)
for i in range(len(network_texts)):
    for j in range(i+1, len(network_texts)):
        if similarity_matrix[i][j] > 0.5:
            G.add_edge(i, j, weight=similarity_matrix[i][j])

# Draw the network
fig, ax = plt.subplots(figsize=(12, 10))

# Layout
pos = nx.spring_layout(G, k=0.5, iterations=50)

# Node colors (categories)
node_colors = []
for i, text in enumerate(network_texts):
    if 'AI' in text or 'agent' in text.lower() or 'learning' in text.lower():
        node_colors.append('#FF6B6B')  # Red for AI/Agents
    else:
        node_colors.append('#4ECDC4')  # Teal for other

# Draw nodes
nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=3000, alpha=0.8, ax=ax)

# Draw edges
edges = G.edges()
if edges:
    weights = [G[u][v]['weight'] for u, v in edges]
    nx.draw_networkx_edges(G, pos, width=[w*3 for w in weights], alpha=0.6, ax=ax)

# Draw labels
labels = {i: network_texts[i][:20] + '...' if len(network_texts[i]) > 20 else network_texts[i]
          for i in range(len(network_texts))}
nx.draw_networkx_labels(G, pos, labels, font_size=8, font_weight='bold', ax=ax)

# Legend
ai_patch = mpatches.Patch(color='#FF6B6B', label='AI/ML/Agents', edgecolor='black')
other_patch = mpatches.Patch(color='#4ECDC4', label='Other', edgecolor='black')
ax.legend(handles=[ai_patch, other_patch], loc='upper left', fontsize=12)

plt.title('Embedding Similarity Network (Threshold > 0.5)', fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

print("\n🔗 Graphique de réseau : Les lignes connectent les textes similaires !")

---
## Step 8: Résumé et conclusions

In [ ]:
print("\n" + "="*80)
print("RÉSUMÉ DE L'APPRENTISSAGE")
print("="*80)
print("""
✓ Tokenisation : Comment les textes sont divisés en unités plus petites
✓ Embeddings : Conversion de texte en vecteurs numériques haute dimension
✓ Similarité sémantique : Les textes similaires ont des embeddings proches
✓ Visualisation : Réduction 2D pour explorer l'espace sémantique

Modèles utilisés (100% open source et gratuit):
- Tokenizer: DistilBERT-base-uncased
- Embeddings: Sentence-Transformers (all-MiniLM-L6-v2)
- Réduction: PCA (scikit-learn)
- Graphique: NetworkX + Matplotlib
""")
print("="*80)

# Statistics summary
print("\n📊 STATISTIQUES FINALES:")
print(f"  - Textes analysés: {len(all_texts)}")
print(f"  - Dimension des embeddings: {len(embeddings[0])}")
print(f"  - Tokenizer: DistilBERT (uncased)")
print(f"  - Model embeddings: Sentence-Transformers (MiniLM-L6-v2)")

---
## Points clés de référence

| Concept | Explication |
|---------|----------|
| **Tokenization** | Processus de division du texte en unités discrètes (tokens) |
| **Token** | Unité individuelle (mot, sous-mot, caractère spécial) |
| **Embedding** | Vecteur numérique représentant le sens du texte |
| **Dimension** | Nombre de valeurs dans le vecteur (ex: 384 pour MiniLM) |
| **Similarité** | Distance entre deux embeddings (cosine similarity) |
| **PCA** | Technique pour réduire les dimensions en préservant la variance |

### Modèles open source recommandés

**Tokenizers :**
- `distilbert-base-uncased` (très léger, 66M paramètres)
- `roberta-base` (plus puissant, 125M paramètres)
- `bert-base-multilingual-cased` (multilingual)

**Embeddings (Sentence-Transformers) :**
- `all-MiniLM-L6-v2` : **Recommandé** (22M params, ultra-rapide)
- `all-mpnet-base-v2` : Plus puissant (109M params)
- `distiluse-base-multilingual-cased-v2` : Multilingual

**Avantages :**
- ✅ Gratuit et sans limites
- ✅ Fonctionnent hors ligne
- ✅ Pas de clés API requises
- ✅ Exécution locale rapide sur GPU/CPU